In [5]:
import pandas as pd
from yt_dlp import YoutubeDL

In [6]:
import os
DOWNLOAD_DIR = './downloads'
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

In [7]:
URL = "https://www.youtube.com/watch?v=LUHGvz8skoo"

In [14]:
class YT():
    def __init__(self, filename, info_dict):
        self.filename = filename
        self.info_dict = info_dict
        self.audio_filepath = self.get_audio_filepath()
        self.caption_filepath = self.get_caption_filepath()

    def get_audio_filepath(self):
        return self.filename + '.mp3'

    def get_caption_filepath(self):
        if self.info_dict.get('requested_subtitles'):
            ext = list(self.info_dict['requested_subtitles'].keys())[0]
            return self.filename + '.' + ext + '.srt'
        else:
            return None

In [9]:
urls = [
    URL
]
ydl_opts = {
    'format': 'bestaudio',
    'writesubtitles': True,
    'subtitlesformat': 'srt',
    'outtmpl': {
        'default': 'downloads/%(title)s.%(ext)s',
    },

    # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
    'postprocessors': [
        {  # Extract audio using ffmpeg
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
        },
        {  # Convert subtitles to srt
            'key': 'FFmpegSubtitlesConvertor',
            'format': 'srt'
        }
    ]
}

In [15]:
with YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(urls[0], download=False)
    filename = ydl.prepare_filename(info_dict).replace(
        '.webm', '').replace('.mp4', '')
    ydl.download(urls)

# Write the yt information as a dataclass, to get information dynamically
yt_file = YT(filename, info_dict)

[youtube] Extracting URL: https://www.youtube.com/watch?v=LUHGvz8skoo
[youtube] LUHGvz8skoo: Downloading webpage
[youtube] LUHGvz8skoo: Downloading ios player API JSON
[youtube] LUHGvz8skoo: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/watch?v=LUHGvz8skoo
[youtube] LUHGvz8skoo: Downloading webpage
[youtube] LUHGvz8skoo: Downloading ios player API JSON
[youtube] LUHGvz8skoo: Downloading m3u8 information
[info] LUHGvz8skoo: Downloading 1 format(s): 251
[info] There are no subtitles for the requested languages
[download] downloads/《刺青》#菜冠雙頭｜#現場漫才｜#現場喜劇.webm has already been downloaded
[download] 100% of    8.35MiB
[ExtractAudio] Destination: downloads/《刺青》#菜冠雙頭｜#現場漫才｜#現場喜劇.mp3
Deleting original file downloads/《刺青》#菜冠雙頭｜#現場漫才｜#現場喜劇.webm (pass -k to keep)
[SubtitlesConvertor] There aren't any subtitles to convert


In [11]:
from faster_whisper import WhisperModel
model_size_or_path = "small"
temperature = 0.001
initial_prompt = "蔡冠雙頭喜劇演員漫才"

/Users/bocheng/Learning/yt-gen-caption/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
model = WhisperModel(model_size_or_path)

[2024-06-20 22:02:07.077] [ctranslate2] [thread 4331661] [warning] The compute type inferred from the saved model is float16, but the target device or backend do not support efficient float16 computation. The model weights have been automatically converted to use the float32 compute type instead.


In [16]:
audio_filepath = yt_file.audio_filepath

segments, _ = model.transcribe(
    audio_filepath,
    vad_filter=True,
    temperature=[temperature],
    initial_prompt=initial_prompt
)

In [17]:
for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

[0.62s -> 2.62s] 蔡冠雙頭 大家好
[6.96s -> 8.96s] 冠冠 我想刺青
[8.96s -> 9.96s] 好啊
[9.96s -> 10.96s] 想問一下你的意見
[10.96s -> 12.96s] 喔 畢竟我有刺青嘛
[12.96s -> 14.96s] 對啊 刺青是一輩子的事耶
[14.96s -> 15.96s] 不能跟你一樣失敗
[16.96s -> 17.96s] 不要這樣說
[17.96s -> 19.96s] 這是USB嗎
[20.96s -> 21.96s] 漫才麥克風啊
[21.96s -> 23.96s] 跟這個一樣的
[23.96s -> 25.96s] 也不是說刺得不好
[26.96s -> 27.96s] 幹嘛刺啊
[28.96s -> 30.96s] 想說刺一下這樣把手舉起來
[30.96s -> 31.96s] 我們這個學生講漫子
[31.96s -> 32.96s] 開小漫才了吧
[32.96s -> 33.96s] 好 那你講刺什麼
[33.96s -> 34.96s] 你再臉
[34.96s -> 35.96s] 開始喔
[36.96s -> 37.96s] 幹嘛刺啊
[37.96s -> 39.96s] 想說刺一下我把你的臉舉起來
[39.96s -> 41.96s] 就可以隨時講漫才
[42.96s -> 43.96s] 開小漫才
[43.96s -> 44.96s] 開小漫才
[44.96s -> 46.96s] 你要刺手掌嗎
[47.96s -> 48.96s] 我這邊還是在臉上
[48.96s -> 49.96s] 兩張臉
[49.96s -> 51.96s] 蔡冠雙頭
[56.68s -> 57.68s] 還是在後腦頭
[57.68s -> 58.68s] 後腦頭
[58.68s -> 59.68s] 為什麼
[59.68s -> 60.68s] 因為之後髮尾流長
[60.68s -> 61.68s] 弄成中分的
[61.68s -> 62.68s] 就跟一下你講
[62.68s -> 63.68s] 什麼
[63.68s -> 65.68s] 想到這邊念自己到的
[65.68s -> 66.68s] 這樣沒幹嘛
[66.68s -> 67.68s] 這樣我就可以
[67.68s ->